In [2]:
import pandas as pd
import sqlite3
from os import listdir

In [1]:
sql_file_path = '/div/pythagoras/u5/vipink/main/data_collection/downloaded_data/SQLDumps/cisbp_2.00.motif_all_test.sql'
db_file_path = '/div/pythagoras/u5/vipink/main/data_collection/downloaded_data/cisbp_2.00.motif_all_test.db'
tfs_db_file_path = '/div/pythagoras/u5/vipink/main/data_collection/downloaded_data/cisbp_2.00.tfs.db'

In [3]:
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()

In [4]:
with open(sql_file_path, 'r') as file:
    sql_script = file.read()
sql_commands = sql_script.split(';')


In [5]:
idx = 0
for command in sql_commands:

    command = command.strip()
    if command:
        try:
            cursor.execute(command)
        except sqlite3.Error as e:
            print(f"Error executing command: {command}")
            print(f"Error message: {e}")
            print(idx)
        idx+=1

In [6]:
conn.commit()
conn.close()

In [7]:
connection = sqlite3.connect(db_file_path)

# Step 2: Create a cursor and execute the query
cursor = connection.cursor()
query = "SELECT MA_ID, Motif_ID, TF_ID, Species, Evidence FROM motif_all;"
cursor.execute(query)

# Step 3: Fetch all TF_Name values
motif_names = cursor.fetchall()

connection.close()

In [9]:
motif_names[0]

('MA00000001_2.0',
 'M01091_2.00',
 'T000021_2.00',
 'Drosophila_melanogaster',
 'D')

In [10]:
len(motif_names)

10923158

In [11]:
connection = sqlite3.connect(tfs_db_file_path)

# Step 2: Create a cursor and execute the query
cursor = connection.cursor()
query = "SELECT TF_Name,TF_Species, TF_ID FROM tfs;"
cursor.execute(query)

# Step 3: Fetch all TF_Name values
tf_names = cursor.fetchall()
connection.close()


In [12]:
print("TF_Name entries:")
name_list = []
species_list = []
cisbp_ID_list = []
for row in tf_names:
    name_list.append(row[0])
    species_list.append(row[1])
    cisbp_ID_list.append(row[2])  # Each row is a tuple with the TF_Name value as the first element

TF_Name entries:


In [13]:
motif_id_list = []
tf_ID_list = []
ma_ID_list = []
species_ID_list = []
evidence_list = []
for row in motif_names:
    ma_ID_list.append(row[0])
    motif_id_list.append(row[1])
    tf_ID_list.append(row[2])
    species_ID_list.append(row[3])
    evidence_list.append(row[4])

    


In [14]:
CISBP_motif_tbl = pd.DataFrame({'ma_ID':ma_ID_list,'motif_ID':motif_id_list,'tf_id':tf_ID_list,'species':species_ID_list,'evidence':evidence_list})
CISBP_TF_tbl = pd.DataFrame({'name':name_list,'species':species_list,'tf_id':cisbp_ID_list})

In [12]:
CISBP_motif_tbl.evidence.value_counts()

evidence
I    10911667
D       11491
Name: count, dtype: int64

In [15]:
CISBP_motif_tbl.query("evidence == 'D'").merge(CISBP_TF_tbl.loc[:,['tf_id','name']],how='left')

,ma_ID,motif_ID,tf_id,species,evidence,name
0,MA00000001_2.0,M01091_2.00,T000021_2.00,Drosophila_melanogaster,D,bnl
1,MA00000002_2.0,M00001_2.00,T000084_2.00,Saccharomyces_cerevisiae,D,ABF1
2,MA00000003_2.0,M00902_2.00,T000084_2.00,Saccharomyces_cerevisiae,D,ABF1
3,MA00000004_2.0,M07432_2.00,T000084_2.00,Saccharomyces_cerevisiae,D,ABF1
4,MA00000005_2.0,M08469_2.00,T000084_2.00,Saccharomyces_cerevisiae,D,ABF1
...,...,...,...,...,...,...
11486,MA10921841_2.0,M02629_2.00,T390654_2.00,Neurospora_crassa,D,NCU08848
11487,MA10921842_2.0,M02630_2.00,T390658_2.00,Neurospora_crassa,D,NCU09033
11488,MA10921843_2.0,M02631_2.00,T390668_2.00,Neurospora_crassa,D,NCU10258
11489,MA10921844_2.0,M01431_2.00,T390671_2.00,Neurospora_crassa,D,NCU10697


In [16]:
CISBP_motif_pwm_folder = "/storage/mathelierarea/raw/CIS-BP/whole_collection/pwms/"

In [17]:
JASPAR_metadata_tbl = pd.read_csv("/storage/mathelierarea/raw/JASPAR/2024/metadata/JASPAR_latest_profiles_metadata.tab",sep="\t")

In [19]:
motifID_name_list = [f.split(".txt")[0] for f in listdir(CISBP_motif_pwm_folder)]

motifID_path_list = [CISBP_motif_pwm_folder + f for f in listdir(CISBP_motif_pwm_folder)]

cisbp_motif_path_df = pd.DataFrame({'motif_ID':motifID_name_list,'path':motifID_path_list})

(CISBP_motif_tbl
.query("evidence == 'D'")
.merge(CISBP_TF_tbl.loc[:,['tf_id','name']],how='left')
.merge(cisbp_motif_path_df)
.assign(species = lambda df_: [i.replace("_"," ")for i in df_.species])
.merge(JASPAR_metadata_tbl.loc[:,['species','tax_group']].drop_duplicates(),how='inner')
.merge(JASPAR_metadata_tbl.loc[:,['matrix_id','name','species']].drop_duplicates(),how='outer')
.assign(not_in_JASPAR = lambda df_: df_.matrix_id.isna())
.query("not_in_JASPAR")
.species.value_counts()
)

species
Homo sapiens                 972
Mus musculus                 926
Arabidopsis thaliana         308
Drosophila melanogaster      153
Neurospora crassa            141
Aspergillus nidulans          97
Caenorhabditis elegans        93
Saccharomyces cerevisiae      80
Oryza sativa                  52
Dictyostelium discoideum      39
Cannabis sativa               30
Ostreococcus tauri            27
Nematostella vectensis        27
Rattus norvegicus             26
Caenorhabditis briggsae       23
Petunia x hybrida             22
Zea mays                      19
Fusarium graminearum          18
Triticum aestivum             16
Histoplasma capsulatum        12
Xenopus laevis                12
Gallus gallus                 12
Schizosaccharomyces pombe     12
Schizophyllum commune         11
Hordeum vulgare                9
Phytophthora ramorum           8
Puccinia graminis              7
Ciona intestinalis             7
Schistosoma mansoni            5
Antirrhinum majus              4
So